In [11]:
import os
import json
from dotenv import load_dotenv

load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

In [2]:
question = """How would you reconcile the apparent contradictions between utilitarian ethics and Kantian deontological ethics when addressing a moral dilemma in a real-world scenario?"""

competitors = []
answers = []
messages = [{"role": "user", "content": question}]

In [4]:
from openai import OpenAI

# Openai Model
openai = OpenAI()

# Gemini Model
gemini = OpenAI(api_key=google_api_key, base_url="https://generativelanguage.googleapis.com/v1beta/openai/")

# Groq Model
groq = OpenAI(api_key=groq_api_key, base_url="https://api.groq.com/openai/v1")

In [5]:
# <-------------------------Running Openai ---------------------------->
model_name = "gpt-4o-mini"
response = openai.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content
competitors.append(model_name)
answers.append(answer)

# <-------------------------Running Gemini ---------------------------->
model_name = "gemini-2.0-flash"
response = gemini.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content
competitors.append(model_name)
answers.append(answer)


# <-------------------------Running Groq ---------------------------->
model_name = "llama-3.3-70b-versatile"
response = groq.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content
competitors.append(model_name)
answers.append(answer)

In [6]:
# Let's bring this together - note the use of "enumerate"

together = ""
for index, answer in enumerate(answers):
    together += f"# Response from competitor {index+1}\n\n"
    together += answer + "\n\n"

In [7]:
judge = f"""You are judging a competition between {len(competitors)} competitors.
Each model has been given this question:

{question}

Your job is to evaluate each response for clarity and strength of argument, and rank them in order of best to worst.
Respond with JSON, and only JSON, with the following format:
{{"results": ["best competitor number", "second best competitor number", "third best competitor number", ...]}}

Here are the responses from each competitor:

{together}

Now respond with the JSON with the ranked order of the competitors, nothing else. Do not include markdown formatting or code blocks."""

In [8]:
judge_messages = [{"role": "user", "content": judge}]

In [9]:
# Judgement time!

openai = OpenAI()
response = openai.chat.completions.create(
    model="o3-mini",
    messages=judge_messages,
)
results = response.choices[0].message.content
print(results)

{"results": ["2", "1", "3"]}


In [12]:
# OK let's turn this into results!

results_dict = json.loads(results)
ranks = results_dict["results"]
for index, result in enumerate(ranks):
    competitor = competitors[int(result)-1]
    print(f"Rank {index+1}: {competitor}")

Rank 1: gemini-2.0-flash
Rank 2: gpt-4o-mini
Rank 3: llama-3.3-70b-versatile
